In [1]:
%matplotlib inline
# -*- coding: utf-8 -*-
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
sys.path.append('library/')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import time
import matplotlib.dates as md
from scipy import fftpack
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import imp  
from sklearn.cluster import KMeans
from sklearn.externals import joblib
from imblearn.over_sampling import ADASYN
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import sklearn.metrics as metrics
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.neighbors import NearestNeighbors
import json
import requests

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn import mixture
import itertools
from scipy import linalg

In [2]:
import NoiseRemover

In [3]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

# Load Data and Pre-Processing

In [4]:
with open("data/Dataset_z_1024_tweets.json") as f:
    content = f.read()
    content = content.replace("][",",")
    D_1024 = json.loads("".join(content))
D_1024[0]

{'archivesource': 'twitter-search',
 'created_at': 'Mon Oct 02 20:34:31 +0000 2017',
 'from_user': 'ajohouse',
 'from_user_id': '113623863',
 'geo_coordinates_0': '0.0',
 'geo_coordinates_1': '0.0',
 'geo_type': '',
 'id': '914951754223144960',
 'iso_language_code': 'en',
 'profile_image_url': 'http://pbs.twimg.com/profile_background_images/611771378/x747d4959c351659f880a3a579e75250.png',
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': "RT @cristinalaila1: 🚨CBS Exec: No Sympathy For Vegas Because 'Country Music Fans Are Often Gun Toting Republicans' #LasVegas #Shooting  htt…",
 'time': '1506976471',
 'to_user_id': ''}

Extract text data and build index

In [5]:
D_1024_text = [] # clean text list
D_1024_original_text = [] # original text list
D_1024_original_index = []
D_1024_clean_index = []
nr = NoiseRemover.NoiseRemover()
index = 0
for record in D_1024:
    D_1024_original_text.append(record["text"])
    cleaned_record = nr.clean_tweet(tweet=record["text"])
    D_1024_original_index.append(int(index))
    if cleaned_record:
        if not isEnglish(cleaned_record):
            index += 1
            continue
        D_1024_text.append(cleaned_record)
        D_1024_clean_index.append(int(index))
    index += 1

In [6]:
len(D_1024_text),len(D_1024_original_text)

(10119, 11750)

In [7]:
D_1024_text[0:10]

['exec sympathy vega music fan often gun tot',
 'pray kill injure concert lord restore dignity human life',
 'mass shoot since',
 'scary interviewer say lady say go die tonight minute listen',
 'support regulate amendment',
 'nobody say anything shoot inside planet hollywood saw horrible',
 'exec sympathy vega music fan often gun tot',
 'scary interviewer say lady say go die tonight minute listen',
 'bank robber kill big still consider attack',
 'la vega shoot least dead mandalay bay hotel']

Create Dataframes

In [8]:
D_1024_df = pd.DataFrame()
D_1024_df["text"] = D_1024_text
D_1024_df["index"] = D_1024_clean_index
D_1024_df = D_1024_df.drop_duplicates(subset='text', keep='first')

In [9]:
D_1024_df.head(10)

,text,index
0,exec sympathy vega music fan often gun tot,0
1,pray kill injure concert lord restore dignity ...,3
2,mass shoot since,4
3,scary interviewer say lady say go die tonight ...,5
4,support regulate amendment,6
5,nobody say anything shoot inside planet hollyw...,7
8,bank robber kill big still consider attack,10
9,la vega shoot least dead mandalay bay hotel,11
11,viral video man call bullet rain,13
14,beard like happen killer need access firearm s...,16


In [10]:
D_1024_df.shape

(1917, 2)

In [11]:
D_1024_original_df = pd.DataFrame()
D_1024_original_df["text"] = D_1024_original_text
D_1024_original_df["index"] = D_1024_original_index
D_1024_original_df.shape

(11750, 2)

In [12]:
D_1024_original_df.head(10)

,text,index
0,RT @cristinalaila1: 🚨CBS Exec: No Sympathy For...,0
1,#LasVegas #shooting https://t.co/x3INvmma9C,1
2,RT @UTHornsRawk: #breaking #LasVegas #shooting...,2
3,RT @ArchbishopDen: Pray for all those killed a...,3
4,RT @MrScottEddy: Mass Shootings in the U.S. si...,4
5,RT @Fuctupmind: Scary\nInterviewer says lady s...,5
6,RT @RamboPepe41391: #LasVegas #GunControlNow #...,6
7,RT @latunesi: Why nobody say anything about th...,7
8,RT @cristinalaila1: 🚨CBS Exec: No Sympathy For...,8
9,RT @Fuctupmind: Scary\nInterviewer says lady s...,9


In [13]:
D_1024_df.to_csv("processed_data/D_1024_text_cleaned.csv")

# Word-Vector Train

In [14]:
D_1024_df['text'][0].split() #tokenize

['exec', 'sympathy', 'vega', 'music', 'fan', 'often', 'gun', 'tot']

In [15]:
D_1024_tk = []
for record in D_1024_df['text']:
    words = record.split()
    D_1024_tk.append(words)

In [16]:
len(D_1024_tk)

1917

Train Word2Vev

In [17]:
model = Word2Vec(D_1024_tk, min_count=2)

In [18]:
print(model)

Word2Vec(vocab=1195, size=100, alpha=0.025)


In [19]:
print(model['word'])

[ -1.98753318e-03  -2.84849294e-03   4.65128990e-03  -3.61922197e-03
   5.77430520e-03   3.38950986e-03   2.21296190e-03  -2.91639124e-03
  -1.07173119e-05  -1.19302643e-03  -2.81763729e-03  -3.28164129e-03
   2.09989492e-03  -2.39595072e-03   3.48241790e-03   3.18637613e-04
  -2.43356987e-03  -1.69140776e-03  -1.19520424e-04  -1.85899471e-03
   8.81350774e-04  -8.90079362e-04   3.95865133e-03  -3.87939205e-03
  -2.15505110e-03  -3.79312248e-03   1.86661084e-03   3.89613933e-03
   1.11513026e-03   2.76603457e-03   7.55743240e-04   1.93203334e-03
  -3.20015568e-03   4.27217223e-03   3.78558435e-03  -3.25154979e-04
  -3.48249800e-03  -2.02505360e-03   2.56710313e-03  -2.12651258e-03
   3.94320395e-03   1.57410931e-03   3.77362547e-03   1.29170087e-03
   4.53572394e-03   2.59701582e-03  -4.78469161e-03  -4.27431150e-05
  -5.23329480e-03   3.74628580e-03   1.60360694e-04  -2.13348214e-03
   3.69966635e-03   1.37502095e-03  -3.68387508e-03  -1.50915550e-03
   6.41140563e-04   2.93016317e-03

# Averaging Sentence

In [20]:
def get_sentence_vector(model,size, sentence):
    featureVec = np.zeros((1,size))
    nwords = 0
    if(len(sentence) == 0):
        return featureVec
    for word in sentence:
        try:
            word_vector = model[word]
            nwords += 1
            #print word_vector
            featureVec = np.add(featureVec, word_vector)
        except:
            pass #Swallow exception
    if (nwords > 0):
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [21]:
sentence_vectors = []
for sentence in D_1024_tk:
    sentence_vector = get_sentence_vector(model,100, sentence)
    sentence_vectors.append(sentence_vector[0])

# GMM Modeling

In [51]:
gmm = mixture.GaussianMixture(n_components=74).fit(sentence_vectors)

In [52]:
gmm_result = gmm.predict(sentence_vectors)

In [54]:
gmm_result.shape

(1917,)

# Export Result

In [55]:
result_df = pd.DataFrame()

In [56]:
result_df["label"] = gmm_result
result_df["clean_text"] = D_1024_df["text"].values
result_df["index"] = D_1024_df["index"].values

In [57]:
result_df.shape

(1917, 3)

In [58]:
D_1024_original_df.shape

(11750, 2)

In [59]:
result_merge_df = pd.merge(result_df,D_1024_original_df, on="index")

In [60]:
result_merge_df.shape

(1917, 4)

In [61]:
result_merge_df.to_csv("processed_data/gmm_74_result.csv")